Battery level study, but only interesting things
====

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
"""
Copy files from RDSF to a local directory so I can read them

"""

from ema import read

read.copy_battery_files()

In [9]:
"""
Create a dataframe of all the battery data

"""

battery_df = read.battery_lvl_df()
battery_df.head()

100%|██████████| 77/77 [00:00<00:00, 422.67it/s]


/home/mh19137/seaco_malaysia/data/battery_dbs/EMADATA_22904_20211209.db failed with Execution failed on sql 'SELECT * FROM Event;': database disk image is malformed
/home/mh19137/seaco_malaysia/data/battery_dbs/EMADATA_20050_20220412.db failed with Execution failed on sql 'SELECT * FROM Event;': database disk image is malformed
/home/mh19137/seaco_malaysia/data/battery_dbs/EMADATA_15082_20220420.db failed with Execution failed on sql 'SELECT * FROM Event;': database disk image is malformed
/home/mh19137/seaco_malaysia/data/battery_dbs/EMADATA_638_20220428.db failed with Execution failed on sql 'SELECT * FROM Event;': database disk image is malformed
/home/mh19137/seaco_malaysia/data/battery_dbs/EMADATA_35272_20220520.db failed with Execution failed on sql 'SELECT * FROM Event;': database disk image is malformed
/home/mh19137/seaco_malaysia/data/battery_dbs/EMADATA_27345_20220615.db failed with Execution failed on sql 'SELECT * FROM Event;': database disk image is malformed
/home/mh1913

,p_id,battery_lvl,delta,charges,discharges
Datetime,,,,,
2021-12-01 08:00:52,15402,99,1 days 08:00:52,5,6
2021-12-01 09:02:53,15402,97,1 days 09:02:53,5,6
2021-12-01 10:04:53,15402,96,1 days 10:04:53,5,6
2021-12-01 11:06:53,15402,93,1 days 11:06:53,5,6
2021-12-01 12:06:59,15402,91,1 days 12:06:59,5,6
